In [94]:
#15671.033865775826
#25331.261779152555
#36249.51835315551
import numpy as np
list3=[2,2,2]
list4=[1,1,1]
np.dot(list3,list4)
for i in range(1000000):
    Start = time.clock()
    
    np.dot(list3,list4)
        
    End =time.clock()
    t=t+(End-Start)
    
print(t*1000,"ms")

36249.51835315551 ms


In [ ]:
#此公式可能會修改list&dict#不使用
def mean(vector=Vector,component=Component,data_dict=Data_dict,table_list=Table_list):    
    valuelist=data_dict[vector].copy()
    del valuelist[table_list.index(component)]#*
    mean=np.mean(valuelist)
    return mean  

#此公式不會修改list&dict
def deviation(vector,component=Component,data_dict=Data_dict,table_list=Table_list):
    valuelist=data_dict[vector]
    deviation=valuelist[table_list.index(component)]-np.mean(vector)
    return deviation
##################################ok##########################################
#該func會改變dict中list的長度
def sim(vector_a,vector_b,component=Component,data_dict=Data_dict,table_list=Table_list):
  
    valuelist_a=data_dict[vector_a]
    valuelist_b=data_dict[vector_b]
    
    denominator_a=\
    np.dot(valuelist_a,valuelist_a)    
    denominator_b=\
    np.dot(valuelist_b,valuelist_b)    
   
    try:   
        numerator=np.dot(valuelist_a,valuelist_b)
    except ValueError:
        tail_connect(body=valuelist_a,tail=table_list) 
        tail_connect(body=valuelist_b,tail=table_list)
        
    sim=numerator/((denominator_a**0.5)*(denominator_b**0.5)) 
    return sim

def sim_evo(vector):
    sim_evo=sim(vector,Vector)
    return sim_evo
##############################################################################
def neighbor_list(Neighbor):
    simlist=list(map(sim_evo,memberlist))#
    memberlist=list(data_dict.keys())  
    neighbor_sim_list=[]
    neighbor_member_list=[] 
    
    for i in range(neighbor):
        del memberlist[simlist.index(max(simlist))]
        del simlist[simlist.index(max(simlist))]
        neighbor_member_list.append(memberlist[simlist.index(max(simlist))])#人名
        neighbor_sim_list.append(simlist[simlist.index(max(simlist))])#其sim值

    return neighbor_sim_list,neighbor_member_list
    
##############################################################################
def predict(vector=Vector,component=Component,\
            data_dict=Data_dict,table_list=Table_list,\
            neighbor_sim_list=Neighbor_sim_list,\
            neighbor_member_list=Neighbor_member_list):
    
    #memberlist=[]
    #simlist=[]
    sigmaupper=0
    sigmalower=0
    
    deviation_list=list(map(deviation,neighbor_member_list))

        sigmaupper=np.dot(deviation_list,neighbor_sim_list)       
        sigmalower=np.sum(neighbor_sim_list)
        #print(sigmaupper)
        #print(sigmalower)
    predict=(sigmaupper/sigmalower)+np.mean(vector)
    return predict


In [95]:
#工作中
N=100 #eval(input("N:"))
def parts(vector,component):
    if vector == "" :
        vector=rd.choice(list(Data_dict.keys()))
    if component == "":
        component_index=rd.randint(0,len(Data_dict[vector])-1)
        component=Table_list[component_index]
    parts=predict(vector,component)-Data_dict[vector][component_index]
    print(component)
    return parts

def me(vector="",component="",n=N):
    sigmame=0
    for i in range(n):        
        sigmame=sigmame+parts(vector,component)
        print(component)
        print(sigmame)
    me=sigmame/n
    return me

me(vector="264539")

NameError: name 'N' is not defined

In [48]:
#優化 update
#87557.57094452929 ms
#87724.01116860237 ms
#84083.91932571067 ms
#84083.91932571067 ms

def update(key,value,data_dict=Data_dict,table_list=Table_list):
    if value not in table_list:
        table_list.append(value)#list中可找到該資料
        
    try:
        tail_connect(data_dict[key],table_list)#不管有沒有去過都先接尾巴
    except KeyError:
        data_dict[key]=[]
        tail_connect(data_dict[key],table_list)#不管是誰都有了最新版尾巴
        
    finally:
        try:          
            initial_and_plus(key,value)#賦值
        except Exception:
            print("此處發生異常")    

#假裝是對的    
    return data_dict,table_list

#############################################################################
#85780.9712099306 ms
#89302.40499567844 ms
def update(key,value,data_dict=Data_dict,table_list=Table_list):
    if value not in table_list:
        table_list.append(value)#list中可找到該資料
        
    if key in data_dict:#舊人
        try :#假裝有去過
            initial_and_plus(key,value)
        except IndexError:#結果沒去過
            print("我沒去過")#無法重現
            tail_connect(data_dict[key],table_list)#3            
            initial_and_plus(key,value)
        except Exception:
            print("此處發生異常")    
    else:  
        #新增新人     
        data_dict[key]=[]
        tail_connect(body=data_dict[key],tail=table_list)#1
        initial_and_plus(key,value)    

#假裝是對的    
    return data_dict,table_list
#################################################################################
#130001.11361003337 ms
#93408.14577514539 ms
def update(key,value,data_dict=Data_dict,table_list=Table_list):
    if key not in data_dict:#新人
        #新增新人     
        #print("新人:",key)
        data_dict[key]=[]
        if value not in table_list:#新地
            table_list.append(value)
            #print("ltn",table_list)
        #這個人沒去過，不判斷
        tail_connect(body=data_dict[key],tail=table_list)#1
        initial_and_plus(key,value)    
    
    else:#舊人
        if value not in table_list:#新地
            table_list.append(value)
            #print("ltn",table_list)
            #必然沒去過，不判斷
            tail_connect(data_dict[key],table_list)#2          
            initial_and_plus(key,value)
            
        else:#舊地
            #判斷            
            if table_list.index(value)+1<=len(data_dict[key]) :#沒去過
                initial_and_plus(key,value) 
            else:#去過
                print("舊人舊地沒去過")#無法重現
                tail_connect(data_dict[key],table_list)#3            
                initial_and_plus(key,value)
#假裝是對的    
    return data_dict,table_list

NameError: name 'Data_dict' is not defined

In [ ]:
#84083.91932571067 ms
#84083.91932571067 ms
def insert_data():
    s=NewYork_Data_list.pop()
    parts= re.split(r'[;,:\s]\s*',s[0])
    #print(parts)
    for i in range(2,len(parts),2):
        sending=([parts[0],parts[1],parts[i],parts[i+1]])
        yield sending
    yield 
    
#82389.8734188042 ms  
#89多
def insert_data():
    s=NewYork_Data_list.pop()
    parts= re.split(r'[;,:\s]\s*',s[0])
    #print(parts)
    try:
        j=2
        while True:
            sending=([parts[0],parts[1],parts[j],parts[j+1]]) 
            #print("sending",sending)
            j+=2  
            yield(sending)
    except IndexError:
        raise
        yield
        

In [ ]:
#此公式可能會修改list&dict
def mean(vector=Vector,component=Component,data_dict=Data_dict,table_list=Table_list):    
    valuelist=data_dict[vector].copy()
    del valuelist[table_list.index(component)]#*
    mean=np.mean(valuelist)
    return mean  
#此公式不會修改list&dict
def deviation(vector,component=Component,data_dict=Data_dict,table_list=Table_list):
    valuelist=data_dict[vector]
    deviation=valuelist[table_list.index(component)]-np.mean(vector)
    return deviation
##################################ok##########################################
def sim(vector_a,vector_b,component=Component,data_dict=Data_dict,table_list=Table_list):
  
    valuelist_a=data_dict[vector_a]
    valuelist_b=data_dict[vector_b]
    #**
    tail_connect(body=valuelist_a,tail=table_list)
    tail_connect(body=valuelist_b,tail=table_list)
        
    valuelist_a[table_list.index(component)]=0
    valuelist_b[table_list.index(component)]=0
        
    numerator=\
    np.dot(valuelist_a,valuelist_b)    
    denominator_a=\
    np.dot(valuelist_a,valuelist_a)    
    denominator_b=\
    np.dot(valuelist_b,valuelist_b)    
    sim=numerator/((denominator_a**0.5)*(denominator_b**0.5)) 

    return sim

def sim_evo(vector):
    sim_evo=sim(vector,Vector)
    return sim_evo
##############################################################################  
def predict(vector=Vector,component=Component,\
            data_dict=Data_dict,table_list=Table_list,\
            neighbor=Neighbor):
    
    #memberlist=[]
    #simlist=[]
    sigmaupper=0
    sigmalower=0
    
    if simlist==[]:
        memberlist=list(data_dict.keys())
        #print("member->",memberlist)
        simlist=list(map(sim_evo,memberlist))#
        #print("simlist->",simlist)
   
    for i in range(neighbor):
        del memberlist[simlist.index(max(simlist))]
        simlist.remove(max(simlist))
        #print (max(simlist))
        #print(deviation(memberlist[simlist.index(max(simlist))]))
        sigmaupper=sigmaupper\
        +deviation(memberlist[simlist.index(max(simlist))])\
        *max(simlist)        
        sigmalower=sigmalower+max(simlist)
        #print(sigmaupper)
        #print(sigmalower)
    predict=(sigmaupper/sigmalower)+mean(vector)
    return predict
